In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

In [ ]:
dir_root = "Data/"
name = "Corn Super Expert"
output_qaa = f"{dir_root}test_topic"
name, dir_root, output_qaa

In [ ]:
df_text = pd.read_csv(f"{dir_root}topic.csv")
df_text.head()
df_text.info()

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("voidful/context-only-question-generator").to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("voidful/context-only-question-generator")

In [ ]:
def generate(number_of_questions, text, model, tokenizer, device):
    inputs = tokenizer(
        text, 
        max_length=256, 
        return_tensors="pt"
    ).to(device)

    questions = []
    for step in range(0,number_of_questions):
        summary_ids = model.generate(
            inputs["input_ids"], 
            num_beams=2, 
            min_length=0, 
            max_length=36, 
            do_sample=True, 
            num_return_sequences=1, 
            no_repeat_ngram_size=3, 
            temperature=0.8)

        question = tokenizer.batch_decode(
            summary_ids, 
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=False)
        
        questions.append(question[0])

    return questions

In [ ]:
step_ratio = 1.0
step = int(len(df_text["text"].values)*step_ratio)
step

In [ ]:
pbar = tqdm(range(0, len(df_text["text"].values)))
qaa = []

for i in pbar:    
    text = df_text["text"].values[i].strip()
    
    questions = generate(
        number_of_questions = 1,
        text = text,
        model = model,
        tokenizer = tokenizer,
        device = device
    )

    for question in questions:
        qaa.append({
            "instruction" : question.strip(), 
            "input" : name, 
            "output" : text})

print(len(qaa))

In [ ]:
df_qaa_duplicated = pd.DataFrame(qaa)
df_qaa_duplicated = df_qaa_duplicated.drop_duplicates()
df_qaa_duplicated.to_json(output_qaa + ".json",orient="records")
df_qaa_duplicated